# Medical Chatbot

In [5]:
import numpy as numpy
import pandas as pd
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


In [2]:
def load_pdf(path):
    loader = PyPDFLoader(path)
    documents = loader.load()
    return documents

def split_data(input_docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    final_documents = splitter.split_documents(input_docs)
    return final_documents




In [4]:
input_data = load_pdf("../data/Medical_book.pdf")
final_documents = split_data(input_data)
embeddings = get_huggingface_embeddings()

c:\Users\Tejas\miniconda3\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create Vectorstore Embeddings using Pinecone

In [ ]:
from langchain_pinecone import PineconeVectorStore,Pinecone

docstore = PineconeVectorStore.from_documents(documents=final_documents,embedding=embeddings,index_name="medical-chatbot",
                                              pinecone_api_key=PINECONE_API_KEY)



In [33]:
retriever = docstore.as_retriever(search_type='similarity',search_kwargs={"k":5})

In [114]:
retriever.invoke("cancer?")

[Document(id='b6c8c966-24e2-4061-9353-83640746f5ae', metadata={'page': 43.0, 'source': '../data/Medical_book.pdf'}, page_content='and other tumors.'),
 Document(id='59c9d595-351f-4044-affb-07d7361de316', metadata={'page': 591.0, 'source': '../data/Medical_book.pdf'}, page_content='cancer tospread to other organs of the body.'),
 Document(id='3cb94595-135e-420f-bea5-011df4766bce', metadata={'page': 593.0, 'source': '../data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 580Breast cancerGEM -0433 to 0624 - B  10/22/03 6:09 PM  Page 580'),
 Document(id='4f2f8e16-7087-4127-8049-84a677e0c210', metadata={'page': 590.0, 'source': '../data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 577Breast cancerGEM -0433 to 0624 - B  10/22/03 6:08 PM  Page 577'),
 Document(id='34b37b41-10b1-4454-bc62-46558b7327be', metadata={'page': 595.0, 'source': '../data/Medical_book.pdf'}, page_content='DRUG THERAPY. Many breast cancers, particularly')]

# Load LLM Model and Create a Prompt

In [68]:
from langchain_openai import OpenAI,ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",temperature=0.1,max_tokens=500,api_key=OPENAI_API_KEY)

In [110]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
            "You are an assistant for question-answering tasks. "
            "Use the following pieces of retrieved context to answer the question."
            "If you don't know the answer or any question is out of context just say that you don't know."
            "Use three sentences maximum and keep the answer concise."
            "\n\n"
            "Context:{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [111]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [115]:
response = rag_chain.invoke({"input" :"What are causes of Kidney Failure?"})
print(response['answer'])

Kidney failure can be caused by various conditions categorized into prerenal, postrenal, and intrarenal factors. Common causes include dehydration, hemorrhage, septicemia, heart failure, liver failure, and urinary tract obstructions such as prostate gland issues or tumors. Additionally, direct kidney disease, injury, or adverse reactions to medications can also lead to kidney failure.
